# Construyendo una red neuronal Paso a Paso

**En esta sesión haremos**:
- Haremos una red neuronal con más de 1 capa oculta.
- Vamos a implementar una clase que contendrá una red Neuronal.

**Notación**:
- Super-índide $[l]$ denota una cantidad asociada a la $l^{th}$ capa. 
    - Ejemplo: $a^{[L]}$ es la $L^{th}$ capa de activación. $W^{[L]}$ y $b^{[L]}$ son los $L^{th}$ parámetros de la capa.
- Super-índide $(i)$ denota una cantidad aosciada al $i^{th}$ registro. 
    - Ejemplo: $x^{(i)}$ es el $i^{th}$ registro de entrenamiento.
- Sub-índice $i$ denota la $i^{th}$ entrada de un vector.
    - Ejemplo: $a^{[l]}_i$ denota la $i^{th}$ entrada de las $l^{th}$ capas de activación).

## 1 - Paquetes

- [numpy](www.numpy.org) Paquete para computación científica.
- [matplotlib](http://matplotlib.org) Paquete para hacer gráficos en Python.

In [8]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2 - Lo que desarrollaremos

Para construir su red neuronal, implementará varias "funciones auxiliares". Estas funciones auxiliares se usarán en la próxima tarea para construir una red neuronal de dos capas y una red neuronal de "L" capas.

Desarrollaremos lo siguiente:

- Incializar parámetros para una red de 2 capas y para extrapolar en una red de "L" capas.
- Se implmentará como es que se realiza el Forward propagation.
- Se calculará el costo.
- Se implementará como es que se realiza el Backward propagation.
- Actualización de parámetros.

## 3 - Inicialización

Haremos funciones de inicialización de parámetros para 2 capas y luego su extención a "L" capas ocultas.

### 3.1 - Para una Red Neural de 2 capas

- El modelo que haremos tendra una estructura: *LINEAL -> RELU -> LINEAL -> SIGMOIDEA*. 
- Se usarán pesos aleatorios para las matrices de pesos. 
- Para le bias se usará "0" como incialización.

In [9]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Entradas:
    n_x -- Tamaño de la capa de entrada
    n_h -- Tamaño de la capa oculta
    n_y -- Tamaño de la capa de salida
    
    Salidas:
    parameters -- Diccionario de python que contiene los parámetros:
                    W1 -- Matriz de pesos de tamaño (n_h, n_x)
                    b1 -- Vector de Bias de tamaño (n_h, 1)
                    W2 -- Matriz de pesos de tamaño (n_y, n_h)
                    b2 -- Vector de Bias de tamaño (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### Inicializacion de Parametros
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [10]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]


### 3.2 - Inicialización para una Red Neuronal de "L" capas

La inicialización para una red neuronal de "L" capas es mas complicado, debido a que hay mas matrices de pesos y vectores de bias.

Recorda que cuando se calcula $W X + b$ en python se desarrolla lo siguiente:

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Cuando $WX + b$ este será:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

Implementación de una red neuronal con "L" capas.

- La estructura del modelo será *[LINEAL -> RELU] $ \times$ (L-1) -> LINEAL -> SIGMOIDEA*. Esto quiere decir que tendrá $L-1$ capas usando la funcion de activación tipo ReLU seguidopor una capa con función de activación sigmoidea.
- Guardaremos $n^{[l]}$, que es el numero de unidades de diferentes capas, en la variable `layer_dims`.

In [11]:
def initialize_parameters_deep(layer_dims):
    """
    Entradas:
    layer_dims -- Lista que contine las dimensiones de cada capa de nuestra red
    
    Salidas:
    parameters -- Diccionario que contiene los parámetros "W1", "b1", ..., "WL", "bL":
                    Wl -- Matriz de pesos de tamaño (layer_dims[l], layer_dims[l-1])
                    bl -- Vector Bias de tamaño (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # Numero de capas de la red

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [12]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


## 4 - Modulo para Forward propagation

### 4.1 - Linear Forward .
Ahora que se han inicializado los parámetros, se realizará el módulo de forward propagation, implementando algunas funciones básicas.

- LINEAL FORWARD
- LINEAL -> ACTIVACION donde la ACTIVACION será ReLU o Sigmoidea. 
- [LINEAL -> RELU] $\times$ (L-1) -> LINEAL -> SIGMOIDEA (Para todo el modelo)

El modulo de linear forward usará las siguientes ecuaciones:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

donde $A^{[0]} = X$. 

Cabe recordar que la representación matemática de esta unidad es $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$

In [13]:
def linear_forward(A, W, b):
    """
    Implementacion de Forward Propagation de una capa.

    Entradas:
    A -- Activaciones de las capas previas o la data de entrada: (tamaño de la capa previa o del numero de registros)
    W -- Matriz de pesos: Arreglo de tamaño (tamaño de la capa actual, Tamaño de la capa anterior)
    b -- Vector Bias, Arreglo de tamaño (tamaño de la capa actual, 1)

    Salidas:
    Z -- La entrada de la funcion de activacion, tambien llamada parámetro de pre-activación 
    cache -- Diccionario que contiene a "A", "W" y "b" ; guardados para el paso de backward propagation
    """
    
    Z = np.dot(W, A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [14]:
from testCases_v2 import * # casos estáticos
from dnn_utils_v2 import * # relu, sigmoidea

In [15]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


### 4.2 - Activación Lineal

Se usará 2 funciones de activación:

- **Sigmoidea**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. Esta función ya se ecnuentra cargada como función `sigmoid`. Esta función devuelve **dos** valores: La activación de valor "`a`" y un "`cache`" que contiene a "`Z`" (que es la que será usada). Para usarla solo se debe hacer: 
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: La fórmula matemática de ReLU es $A = RELU(Z) = max(0, Z)$. Esta función tambien se encuentra carga con el nombre de función `relu`. Esta función devuelve **dos** valores: El valor de activación "`A`" y el "`cache`" que contiene a "`Z`" (que es la que será usada). Para usarlo solo se debe hacer:
``` python
A, activation_cache = relu(Z)
```

Por conveniencia se agruparán las dos funciones en una sola funcion (LINEAL->ACTIVACION).

Ahora implementaremos el forward propagation de la capa *LINEAL->ACTIVACION* layer, cuya relación matemática es: $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ donde la activación "g" podrá ser sigmoid() o relu().

In [16]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Entradas:
    A_prev -- Activaciones de la la anterior capa o la data de entrada: (tamaño de la capa anterior, número de registros)
    W -- Matriz de pesos: Arreglo de tamaño (tamaño de la capa actual, tamaño de la capa anterior)
    b -- Vector Bias, arreglo de tamaño (tamaño de la capa actual, 1)
    activation -- el tipo de activación que se usará: "sigmoid" o "relu"

    Salidas:
    A -- La salida de la función de activación, también llamado valor de post-activation 
    cache -- Diccionario que contiene a "linear_cache" y "activation_cache" para 
             el paso de backward
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [ ]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Note**: In deep learning, the "[LINEAR->ACTIVATION]" computation is counted as a single layer in the neural network, not two layers. 

### d) Modelo con "L" Capas 

Como ya implementamos las activaciones, ahora usaremos las mismas funciones para usarlas de la siguiente manera: `linear_activation_forward` con RELU $L-1$ veces, a las que le seguira una capa `linear_activation_forward` con SIGMOID.

PAra el código, la variable `AL` será $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$. (Tambien llamada `Yhat`, i.e., este es $\hat{Y}$.) 

**Desarrollo**:
- Usaremos las funciones previamente escritas.
- En un loop usaremos [LINEAR->RELU] (L-1) veces.

In [17]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """

    Entradas:
    X -- data, de tamaño (numero de variables, número de registros)
    parameters -- salida de initialize_parameters_deep()
    
    Returns:
    AL -- Ultimo valor de post-activación
    caches -- lista de caches que contiene:
                 cada caché de linear_relu_forward() (hay L-1 de ellos, indexados desde 0 a L-2)
                 el caché de linear_sigmoid_forward() (hay uno, indexado como L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # numero de capas en la red neuronal
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev,
                                             parameters['W{}'.format(l)],
                                             parameters['b{}'.format(l)],
                                             'relu')
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A,
                                          parameters['W{}'.format(L)],
                                          parameters['b{}'.format(L)],
                                          'sigmoid')
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [18]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]
Length of caches list = 2


Ahora que ya tenemos a el forward propagation desarrolaldo completamente este toma como entrada un input X y salidas vectoriales $A^{[L]}$ que continen las predicciones. Así mismo tenemos valores intermedios en "chachés". Entonces usando $A^{[L]}$, se podrá calcular el costo de nuestras predicciones.

## 5 - Función de Costo

Ahora se implementará el forward y backward propagation. PAra esto, se necesita calcular el costo, ya que asi podremos saber si el modelo efectivamente esta aprendiendo.

**Desarrollo**: Se calculará el costo tipo cross-entropy $J$, usando la siguiente formula: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


In [20]:
def compute_cost(AL, Y):
    """
    Funcion de calculo de costo.

    Entradas:
    AL -- Vector de probabilidades correspondiente la mis predicciones reales  (1, numero de registros)
    Y --  Vector de "Etiquetas" verdaderas (Pro ejemplo: 0 si no es un gato, 1 si es un gato) (1, numero de registros)

    Salidas:
    costo -- costo tipo cross-entropy
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [21]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


## 6 - Modulo de Backpropagation

Tal cual el modulo de forward propagation, se implementaran las funciones necesarias para el back propagation. Recordar que el backpropagation es usado para calcular la gradiente de la funcion de costo con sus respectivos parámetros. 

Ahora, similar al forward propagation, se harán los siguientes pasos:
- LINEAR backward
- LINEAR -> ACTIVATION backward donde ACTIVATION calcula la derivativa de o la funcion de activación Sigmoid o ReLU.
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward

### 6.1 - Linear backward

Para $l$ capas, la parte lineal será: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (seguido por su activación).

Supongamos que ya se tiene calculado la derivada de  $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. Lo que queremos obtender será $(dW^{[l]}, db^{[l]} dA^{[l-1]})$.

Las tres salidas serán $(dW^{[l]}, db^{[l]}, dA^{[l]})$ que son calculados usando la entrada $dZ^{[l]}$. Estas será las formulas que usaremos:
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


In [ ]:
def linear_backward(dZ, cache):
    """
    Se implementará la porción lineal del backward porpagation para una sola capa (capa l)

    Entradas:
    dZ -- Gradiente del costo respecto a la salida lineal (capa actual l)
    cache -- Tupla de valores (A_prev, W, b) tomados desde el forward propagation en la capa actual

    Salidas:
    dA_prev -- Gradiente del costo con respecto a su activación (de la capa previa l-1), con mismo tamaño de A_prev
    dW -- Gradiente del costo con respecto a W (capa actual l), con el mismo tamaño de W
    db -- Gradient del costo con respecto a b (capa actual l), con el mismo tamaño de b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]


    dW = 1 / m * np.dot(dZ, A_prev.T)
    db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
     
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [ ]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


### 6.2 - Linear-Activation backward

Ahora, crearemos la función que une dos funciones auxiliares:
**`linear_backward`** y el paso de activación backward 
**`linear_activation_backward`**. 

Se implementará `linear_activation_backward`, usando las funciones dadas:
- **`sigmoid_backward`**: Implementa el backward propagation para unidades sigmoides. Uso:

```python
dZ = sigmoid_backward(dA, activation_cache)
```

- **`relu_backward`**: Implementa el backward propagation para unidades ReLU. Uso:

```python
dZ = relu_backward(dA, activation_cache)
```

Si $g(.)$  es la función de activación, 
`sigmoid_backward` y `relu_backward` calculan  $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$.  


In [ ]:
def linear_activation_backward(dA, cache, activation):
    """
    Se implementará la capa LINEAR->ACTIVATION.
    
    Entradas:
    dA -- Gradiente de post-activación para la capa actual layer l 
    cache -- Tupla de valores (linear_cache, activation_cache) para el cálculo de backward propagation
    activation -- Es la activación usada para esta capa: "sigmoid" o "relu"
    
    Salidas:
    dA_prev -- Gradiente del costo con respecto a la activación (de la capa anterior l-1), con el mismo tamañao A_prev
    dW -- Gradiente del costo con respecto a W (de la misma capa l), con el mismo tamaño que W
    db -- Gradiente del costo con respecto a b (de la misma capa l), con el mismo tamaño que b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
    
    # Shorten the code
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [ ]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


### 6.3 - Backward para un modelo de "L" capas 

Ahora se implementará el total de la red neuronal. Ya se implementó la funcion `L_model_forward`, donde en cada iteración, se almacena el caché que contiene (X,W,b, y z). En el módulo de backpropagation, se usarán esas variables para calcular las gradientes. Entonces, en la función `L_model_backward`, se iterarán todas las capas ocultas, empezando desde la capa $L$. En cada paso, se usarán los valores caché de la capa $l$ para "backpropagar" a la capa $l$.

** Inicio**:
Para hacer backpropagation, se sabe que la salida es
$A^{[L]} = \sigma(Z^{[L]})$. Entonces el codigo que necesitamos colocar es `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$.
Se usará esta fórmula (resultado de la derivada usando cálculo matemático):
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```

Ahora, podremos usar la gradiente de post-activación `dAL` para continuar con el backward, entonces podremos usar `dAL` en la funcion implementada LINEAR->SIGMOID backward (L_model_forward function). Luego de esto, usaremos un loop para iterar cada una de las capas usando la funcion LINEAR->RELU backward.
Se usarán todos los datos guardados dA, dW, y db de el diccionario de gradientes. Entonces usaremos la fórmula:

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

Por ejemplo, para $l=3$ este guardará $dW^{[l]}$ en `grads["dW3"]`.

Implementaremos backpropagation para *[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* model.

In [ ]:
def L_model_backward(AL, Y, caches):
    """
    Entradas:
    AL -- Vector de probabilidades, salida del forward propagation (L_model_forward())
    Y -- Vector de "etiquetas" verdaderas
    caches -- lista de chaches que contienen:
                cada caché de linear_activation_forward() con "relu" (sus caches[l], desde l en range(L-1) i.e l = 0...L-2)
                el caché de linear_activation_forward() con "sigmoid" (sus caches[L-1])
    
    Salidas:
    grads -- Diccionario de gradientes
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # numero de capas
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # Lueog de esta linea, Y es del mismo tamañao que AL
    
    # Inicializando el backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradientes. Entradas: "AL, Y, caches". Salidas: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,
                                                                                                  current_cache,
                                                                                                  "sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradientes.
        # Entradas: "grads["dA" + str(l + 2)], caches". Salidas: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA{}".format(l + 2)],
                                                                    current_cache,
                                                                    "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [ ]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


### 6.4 - Actualización de Parámetros

Usaremos la gradiente descendente para actualizar los parámetros

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

donde $\alpha$ es el factor de aprendizaje. Luego de calcular los parámetros actualizados, se guardan dentro del diccionario de parámetros.

Se implementará `update_parameters()` para actualizar los parámetros usando gradiente descendente.

Parámetros a actualizar: $W^{[l]}$ y $b^{[l]}$ desde $l = 1, 2, ..., L$. 


In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Actualizar paramtros usando gradiente descendente
    
    Entradas:
    parameters -- diccionario que contiene parámetros
    grads -- diccionario que contiene gradientes, salida de L_model_backward
    
    Salidas:
    parameters -- diccionario que contiene los parametros actualizados 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # numero de capas en la red neuronal

    # Regla de actualización para cada parámetros.
    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
        
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
